<a href="https://colab.research.google.com/github/jjjkkkjjj/PyTorch_TrackNet/blob/master/demo/training-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd drive/My\ Drive/PyTorch_TrackNet/demo
!ls

/content/drive/My Drive/PyTorch_TrackNet/demo
pytorch-xla-env-setup.py
torch-nightly+20200325-cp36-cp36m-linux_x86_64.whl
torchvision-nightly+20200325-cp36-cp36m-linux_x86_64.whl
torch_xla-nightly+20200325-cp36-cp36m-linux_x86_64.whl
training-colab.ipynb
training.ipynb


In [0]:
%matplotlib inline
import sys
sys.path.append('../')
from models.tracknet import TrackNet
from models.train import Trainer
from models.loss import CrossEntropy
from data.datasets import AllTrackNetTennis
from data.transforms import GaussianHeatMap, LossHeatMap

from torch.utils.data import DataLoader
from torchvision.transforms import *
from torch.optim.adadelta import Adadelta

In [4]:
# Note that when you use google-colab, you should set Config.DATA_ROOT manually (set parent directory of 'tennis_tracknet')
"""
directory structure

---PyTorch_TrackNet
 |   |
 |    --demo --- training-colab.ipynb
 |
 --data
    |
    --tennis_tracknet

"""
import os
from data.datasets import Config
print(Config.DATA_ROOT)
Config.DATA_ROOT = os.path.abspath('../../data')
print(Config.DATA_ROOT)

/root/data
/content/drive/My Drive/data


In [5]:
# TPU settings but too slow...
"""
VERSION = "20200325"  #@param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION
"""

'\nVERSION = "20200325"  #@param ["1.5" , "20200325", "nightly"]\n!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py\n!python pytorch-xla-env-setup.py --version $VERSION\n'

In [6]:
transform = Compose([
    Resize((360, 640)),
    ToTensor()
])
target_transform = Compose([
    GaussianHeatMap((360, 640), sigma2=10, threshold=128),
    LossHeatMap(256)
])

train_dataset = AllTrackNetTennis(seq_num=3, transform=transform, target_transform=target_transform)
train_loader = DataLoader(train_dataset,
                          batch_size=2,
                          shuffle=True)

model = TrackNet(image_shape=(360, 640, 3), seq_num=3, batch_norm=True)
print(model)

TrackNet(
  (tracknet_layers): ModuleDict(
    (conv1_1): Conv2d(9, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1_1): ReLU(inplace=True)
    (conv1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1_2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1_2): ReLU(inplace=True)
    (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (conv2_1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2_1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2_1): ReLU(inplace=True)
    (conv2_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2_2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2_2): ReLU(inplace=True)
    (pool2): MaxPool2d(

In [0]:
#import torch_xla
#import torch_xla.core.xla_model as xm
#device = xm.xla_device()
import torch
device = torch.device('cuda')

loss_func = CrossEntropy()
optimizer = Adadelta(model.parameters(), lr=1.0)
trainer = Trainer(model, loss_func=loss_func, optimizer=optimizer, scheduler=None, device=device, log_interval=10)

from models.graph import LiveGraph

trainer.train(1000, train_loader, savemodelname='tracknet', checkpoints_epoch_interval=50, live_graph=LiveGraph((0, 0.05)), max_checkpoints=2)

Training... Epoch: 1, Iter: 258,	 [516/19835	 (3%)]	Loss: 0.018525